In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

import scipy

In [2]:
import seaborn as sns

In [3]:
import sys
sys.path.append('..')

from Approximators.Bernstein import CauchySimplex

In [4]:
from utils import solve_multiple_coefficient_bessel

Consider Bessel's differential equation with initial conditions $y(0)=y(1)=0$. Let $J_m(x)$ and $Y_m(x)$ be Bessel functions of the first and second kind, respectively. Then, this differential equation has solutions of the form
\begin{align}
    y = c_1J_m(\sqrt{\lambda} x) + c_2Y_m(\sqrt{\lambda} x),
\end{align}
for constants $c_1, c_2$. To satisfy the initial conditions, we must have that $c_2=0$ and $J_m(\sqrt{\lambda})=0$. Thus, the eigenvalues of Bessel's differential equation are the square of the roots of the $m$-th Bessel's function, $J_m$.

Take the parameterization $x=e^z-1$. Then this differential equation can be rewritten as
\begin{align}
    (1-e^{-z})^2y''(z)\ +\ e^{-z}(1-e^{-z})y'(z)\ +\ (\lambda(e^z-1)^2-m^2)y(z)\ =\ 0,
\end{align}
with initial conditions $y(0)=y(\ln(2))=0$.

Take the parameterization $x=e^{az}-1$. Then this differential equation can be rewritten as
\begin{align}
    \frac{1}{a^2}(1-e^{-az})^2y''(z)\ +\ \frac{1}{a}e^{-az}(1-e^{-az})y'(z)\ +\ (\lambda(e^{az}-1)^2-m^2)y(z)\ =\ 0,
\end{align}
with initial conditions $y(0)=y(\ln(2)/a)=0$.


In [5]:
m = 2
a = 1

In [6]:
x = np.linspace(0, np.log(2) / a, 256)
y = [(1 - np.exp(-a * x)) ** 2 / (a ** 2), np.exp(-a * x) * (1 - np.exp(-a * x)) / a, 
     (np.exp(a * x) - 1) ** 2]

In [7]:
n_eigenvals = 20

In [8]:
n_vals = np.arange(4, 20 + 1, 1)

In [9]:
true_eigenvalues = scipy.special.jn_zeros(m, n_eigenvals) ** 2

In [10]:
Nx = 2 ** 12
Nx

4096

# Polynomial Approximation

In [11]:
polynomial_results = []

In [12]:
for n in n_vals:
    print(f"Starting n = {n}")

    polynomial_approximator = CauchySimplex(n, 0).fit(x, y)
    
    time_taken, solver = solve_multiple_coefficient_bessel(polynomial_approximator, a, m, Lx=0, Ux=np.log(2)/a, 
                                                           Nx=Nx, dtype=np.float64, n_eigenvals=n_eigenvals, 
                                                           n_runs=5)
    
    evals = np.sort(solver.eigenvalues.real)
    
    y_pred = polynomial_approximator(x)
    approximation_error = np.mean([np.linalg.norm(y1 - y2) for (y1, y2) in zip(y_pred, y)])

    ratio_dataframe = pd.DataFrame(abs(evals - true_eigenvalues), columns=['Eigenvalue Errors'])
    ratio_dataframe['Approximator'] = 'Polynomial'
    ratio_dataframe['Approximation Error'] = approximation_error
    ratio_dataframe['Num. Coefs'] = n
    ratio_dataframe['Time'] = time_taken
    
    polynomial_results.append(ratio_dataframe)

Starting n = 4
2023-10-05 12:42:51,248 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.3e+00/s
Starting n = 5
2023-10-05 12:42:52,648 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.2e+00/s
Starting n = 6
2023-10-05 12:42:54,096 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.1e+00/s
Starting n = 7
2023-10-05 12:42:55,780 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 8.6e-01/s
Starting n = 8
2023-10-05 12:42:57,496 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 8.8e-01/s
Starting n = 9
2023-10-05 12:42:59,279 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 8.0e-01/s
Starting n = 10
2023-10-05 12:43:01,140 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, R

# Rational Approximation

In [13]:
rational_results = []

In [14]:
for n in n_vals:
    print(f"Starting n = {n}")

    rational_approximator = CauchySimplex(n, n, hot_start=True, max_iter=500).fit(x, y)
    
    time_taken, solver = solve_multiple_coefficient_bessel(rational_approximator, a, m, Lx=0, Ux=np.log(2)/a, 
                                                           Nx=Nx, dtype=np.float64, n_eigenvals=n_eigenvals, 
                                                           n_runs=5)
    
    evals = np.sort(solver.eigenvalues.real)
    
    y_pred = rational_approximator(x)
    approximation_error = np.mean([np.linalg.norm(y1 - y2) for (y1, y2) in zip(y_pred, y)])

    ratio_dataframe = pd.DataFrame(abs(evals - true_eigenvalues), columns=['Eigenvalue Errors'])
    ratio_dataframe['Approximator'] = 'Rational'
    ratio_dataframe['Approximation Error'] = approximation_error
    ratio_dataframe['Num. Coefs'] = n
    ratio_dataframe['Time'] = time_taken
    
    rational_results.append(ratio_dataframe)

Starting n = 4
2023-10-05 12:43:22,032 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.2e+00/s
Starting n = 5
2023-10-05 12:43:23,399 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.2e+00/s
Starting n = 6
2023-10-05 12:43:25,010 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 1.0e+00/s
Starting n = 7
2023-10-05 12:43:26,621 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 9.5e-01/s
Starting n = 8
2023-10-05 12:43:28,371 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 8.4e-01/s
Starting n = 9
2023-10-05 12:43:30,172 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, Remaining: 0s, Rate: 8.2e-01/s
Starting n = 10
2023-10-05 12:43:32,111 subsystems 0/1 INFO :: Building subproblem matrices 1/1 (~100%) Elapsed: 1s, R

# Results

In [15]:
results_df = pd.concat(polynomial_results + rational_results)

In [16]:
results_df.head()

,Eigenvalue Errors,Approximator,Approximation Error,Num. Coefs,Time
0,0.002180,Polynomial,0.000807,4,0.304829
1,0.024970,Polynomial,0.000807,4,0.304829
2,0.052212,Polynomial,0.000807,4,0.304829
3,0.017421,Polynomial,0.000807,4,0.304829
4,0.115873,Polynomial,0.000807,4,0.304829


In [17]:
average_error = results_df.groupby(['Num. Coefs', 'Approximator']).mean()['Eigenvalue Errors'].copy()
average_error.name = 'Eigenvalue Error'

time_taken = results_df.groupby(['Num. Coefs', 'Approximator']).mean()['Time']
time_taken.name = 'Time (sec)'

In [18]:
approximation_error = results_df.groupby(['Num. Coefs', 'Approximator']).mean()['Approximation Error']

In [19]:
results = pd.concat([average_error, approximation_error, time_taken], axis=1).reset_index()
results = results.pivot_table(index='Num. Coefs', columns='Approximator')

In [20]:
column_order = [(col_name, approximator_type) 
                for col_name in ['Eigenvalue Error', 'Approximation Error', 'Time (sec)']
                for approximator_type in ['Polynomial', 'Rational']]

In [21]:
results = results.loc[:, column_order]

In [22]:
results

Eigenvalue Error               Approximation Error                \
Approximator       Polynomial      Rational          Polynomial      Rational   
Num. Coefs                                                                      
4                1.143105e+02  8.514702e-03        8.065302e-04  6.459914e-06   
5                2.306063e-01  4.709668e-04        4.803097e-05  8.522387e-08   
6                1.569276e-02  1.711326e-06        2.418464e-06  1.058997e-09   
7                4.490605e-04  1.768933e-09        1.058366e-07  1.695614e-12   
8                1.292217e-05  3.935522e-10        4.102922e-09  3.830889e-13   
9                2.260724e-07  1.477598e-10        1.428941e-10  2.326859e-14   
10               1.219829e-08  4.504965e-11        4.519707e-12  1.344817e-14   
11               2.553179e-09  2.930995e-08        1.310151e-13  1.950053e-14   
12               2.507138e-09  1.497094e-07        5.571855e-15  5.279098e-14   
13               2.494897e-09  3.479149e-08        3.954182e-15  2.519539e-14   
14               2.522943e-09  7.197956e-08        4.065601e-15  2.364888e-13   
15               2.458164e-09  5.454224e-08        2.273765e-14  3.582096e-13   
16               2.410553e-09  5.055905e-08        3.175847e-14  9.582135e-13   
17               2.389374e-09  6.925822e-08        3.945272e-14  1.340202e-12   
18               2.444482e-09  5.329954e-08        1.634799e-14  8.505014e-13   
19               2.498570e-09  1.529129e-08        1.704999e-14  2.165451e-12   
20               2.435204e-09  2.308255e-08        2.374595e-14  1.499045e-12   

             Time (sec)            
Approximator Polynomial  Rational  
Num. Coefs                         
4              0.304829  0.264304  
5              0.275639  0.267124  
6              0.285946  0.300395  
7              0.344226  0.315251  
8              0.329700  0.341809  
9              0.354465  0.349608  
10             0.369046  0.377657  
11             0.384664  0.392696  
12             0.376545  0.402900  
13             0.388549  0.424211  
14             0.375645  0.432685  
15             0.398776  0.449777  
16             0.388566  0.523832  
17             0.390876  0.511815  
18             0.396684  0.562794  
19             0.383640  0.580423  
20             0.374358  0.616725

In [23]:
formatters = [lambda x: f"{x:.4e}"] * 4\
                + [lambda x: f"{x:.4f}"] * 2
print(results.to_latex(formatters=formatters))

\begin{tabular}{lrrrrrr}
\toprule
 & \multicolumn{2}{r}{Eigenvalue Error} & \multicolumn{2}{r}{Approximation Error} & \multicolumn{2}{r}{Time (sec)} \\
Approximator & Polynomial & Rational & Polynomial & Rational & Polynomial & Rational \\
Num. Coefs &  &  &  &  &  &  \\
\midrule
4 & 1.1431e+02 & 8.5147e-03 & 8.0653e-04 & 6.4599e-06 & 0.3048 & 0.2643 \\
5 & 2.3061e-01 & 4.7097e-04 & 4.8031e-05 & 8.5224e-08 & 0.2756 & 0.2671 \\
6 & 1.5693e-02 & 1.7113e-06 & 2.4185e-06 & 1.0590e-09 & 0.2859 & 0.3004 \\
7 & 4.4906e-04 & 1.7689e-09 & 1.0584e-07 & 1.6956e-12 & 0.3442 & 0.3153 \\
8 & 1.2922e-05 & 3.9355e-10 & 4.1029e-09 & 3.8309e-13 & 0.3297 & 0.3418 \\
9 & 2.2607e-07 & 1.4776e-10 & 1.4289e-10 & 2.3269e-14 & 0.3545 & 0.3496 \\
10 & 1.2198e-08 & 4.5050e-11 & 4.5197e-12 & 1.3448e-14 & 0.3690 & 0.3777 \\
11 & 2.5532e-09 & 2.9310e-08 & 1.3102e-13 & 1.9501e-14 & 0.3847 & 0.3927 \\
12 & 2.5071e-09 & 1.4971e-07 & 5.5719e-15 & 5.2791e-14 & 0.3765 & 0.4029 \\
13 & 2.4949e-09 & 3.4791e-08 & 3.9542e-15